<a href="https://colab.research.google.com/github/TemrinaDarya/turbines-2022/blob/hw-5/TemrinaDarya_hw5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Оценить расходы в цилиндр высокого давления и в конденсатор турбины без промежуточного перегрева пара.

Заданные параметры:

1.  Номинальная электрическая мощность: 250 МВт
2.  Начальное давление свежего пара: 12.8 МПа
1.  Начальная температура свежего пара: 540 C
2.  Конечное давление пара: 6.9 кПа
1.  Температура питательной воды: 263 C
2.  Число отборов: 8
1.  Механическое КПД: 99.2%
2.  КПД Электрогенератора: 99%


In [ ]:
!pip install iapws

from iapws import IAPWS97 as gas

MPa = 10 ** 6
kPa = 10 ** 3
unit = 1 / MPa
to_kelvin = lambda x: x + 273.15 if x else None

In [2]:
electrical_power = 250 * (10 ** 6)
p0 = 12.8 * MPa
t0 = 540
p_middle = 2.49 * MPa
t_middle = 540
pk = 6.9 * kPa
t_feed_water = 263
p_feed_water = 1.4 * p0
z = 8

internal_efficiency = 0.85
mechanical_efficiency = 0.992
generator_efficiency = 0.99

In [3]:
delta_p0 = 0.05 * p0
real_p0 = p0 - delta_p0
point_00 = gas(P=p0 * unit, T=to_kelvin(t0))
point_0 = gas(P=real_p0 * unit, h=point_00.h)
point_feed_water = gas(P=p_feed_water * unit, T=to_kelvin(t_feed_water))
point_k0 = gas(P=pk * unit, s=point_00.s)
H_0 = point_0.h - point_k0.h
H_i = H_0 * internal_efficiency
h_k = point_0.h - H_i
point_k = gas(P=pk * unit, h=h_k)
point_k_water = gas(P=pk * unit, x=0)

T_s = gas(P=p0 * unit, x=0).T
Ksi = (point_feed_water.T - point_k.T) / (T_s - point_k.T)


In [4]:
def count_ksi(point_0, point_k, point_k_water, point_feed_water):
    numerator_without = (point_00.s - point_k_water.s) * point_k.T
    denumerator_without = point_0.h - point_k_water.h
    without_part = 1 - (numerator_without / denumerator_without)

    numerator_infinity = (point_00.s - point_feed_water.s) * point_k.T
    denumerator_infinity = point_0.h- point_feed_water.h
    infinity_part = 1 - (numerator_infinity / denumerator_infinity)

    ksi_infinity = 1 - (without_part / infinity_part)
    ksi = 0.93 * ksi_infinity

    return ksi


def count_inlet_mass_flow(electrical_power, estimated_heat_drop, mechanical_efficiency, generator_efficiency):
    inlet_mass_flow = electrical_power / (estimated_heat_drop * 1000 * mechanical_efficiency * generator_efficiency)
    return inlet_mass_flow


def count_condenser_mass_flow(efficiency, electrical_power, point_k, point_k_water):
    condenser_mass_flow = (electrical_power / (
            (point_k.h - point_k_water.h) * 1000 * mechanical_efficiency * generator_efficiency) * (
                                   (1 / efficiency) - 1))
    return condenser_mass_flow


def count_inlet_and_condenser_mass_flow(point_0, point_k, point_k_water, point_feed_water, H_i):
    ksi = count_ksi(point_0, point_k, point_k_water, point_feed_water)
    eff_denum = point_0.h - point_k_water.h

    efficiency = (H_i / eff_denum) * (1 / (1 - ksi))
    estimated_heat_drop = efficiency * (point_0.h - point_feed_water.h)
    inlet_mass_flow = count_inlet_mass_flow(electrical_power, estimated_heat_drop, mechanical_efficiency,
                                            generator_efficiency)
    condenser_mass_flow = count_condenser_mass_flow(efficiency, electrical_power, point_k, point_k_water)
    return inlet_mass_flow, condenser_mass_flow

In [ ]:
inlet_mass_flow, condenser_mass_flow = count_inlet_and_condenser_mass_flow(
    point_0, point_k, point_k_water, point_feed_water, H_i)

print("Массовый расход в турбину на входе", inlet_mass_flow)
print("Массовый расход в конденсатор:", condenser_mass_flow)
